In [1]:
import os
# Changing the working directory
os.chdir('../..')
os.getcwd()

'/Users/javiermunoz/Universidad/MasterDSyBD/TFM/TFM_DSBD'

In [2]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

import tensorflow as tf
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc("figure", figsize=(15, 5))

import numpy as np
import pandas as pd

In [3]:
gpu_devices

[]

In [4]:
leaderboard = pd.read_csv("data/leaderboard.csv")
leaderboard

,model_type,model_name,train_acc,val_acc,test_acc,train_loss,val_loss,test_loss,epoch,path_to_model,subset_size
0,convlstm,simple,0.96,0.93,0.93,0.15,0.20,0.19,39.0,src/convlstm/bestmodels/best_simple_39_0.93_10...,10.0
1,convlstm,stateless,0.93,0.95,0.91,0.22,0.19,0.19,22.0,src/convlstm/bestmodels/best_stateless_22_0.95...,10.0
2,mediapipe,mp,0.90,0.90,0.87,0.27,0.35,0.54,178.0,src/mediapipe/bestmodels/best_mp_178_0.90_mode...,20.0
3,mediapipe,mp,0.89,0.85,0.85,0.38,0.62,0.43,159.0,src/mediapipe/bestmodels/best_mp_159_0.85_mode...,40.0
4,mediapipe,mp,0.93,0.86,0.86,0.24,0.63,0.55,167.0,src/mediapipe/bestmodels/best_mp_167_0.86_mode...,30.0
5,3DCNN,3dcnn,0.93,0.92,0.91,0.33,0.37,0.40,132.0,src/3DCNN/bestmodels/best_3dcnn_132_0.92_model.h5,20.0
6,convlstm,stateless,0.90,0.94,0.92,0.27,0.24,0.22,63.0,src/convlstm/bestmodels/best_stateless_63_0.94...,20.0
7,convlstm,stateless,0.95,0.92,0.82,0.20,0.26,0.46,93.0,src/convlstm/bestmodels/best_stateless_91_0.92...,30.0
8,convlstm,stateless,0.87,0.77,0.74,0.38,1.05,1.06,104.0,src/convlstm/bestmodels/best_stateless_104_0.7...,40.0
9,convlstm,simple,0.83,0.65,0.65,0.48,1.36,1.27,170.0,src/convlstm/bestmodels/best_simple_170_0.65_m...,40.0


In [5]:
path_to_test = leaderboard['path_to_model'][0]

model_to_test = tf.keras.models.load_model(path_to_test)
model_to_test.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, None, 224, 224, 3  0         
                             )                                   
                                                                 
 conv_lstm2d (ConvLSTM2D)    (None, None, 222, 222, 8  3200      
                             )                                   
                                                                 
 batch_normalization (BatchN  (None, None, 222, 222, 8  32       
 ormalization)               )                                   
                                                                 
 dropout (Dropout)           (None, None, 222, 222, 8  0         
                             )                                   
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, 220, 220, 16)      1

In [6]:
from src.utils import load_dict
subset = 10
labels=load_dict(f'data/subset_{subset}_lsa_64/pickl_files/labels_map.pkl')
labels

FileNotFoundError: [Errno 2] No such file or directory: 'data/subset_10_lsa_64/pickl_files/labels_map.pkl'

In [8]:
import mediapipe as mp
from numpy.random import default_rng
from src.config import LM_PER_VIDEO
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils 
from src.mp_funs import extract_landmarks_to_np

import cv2

threshold = 0.6

cap = cv2.VideoCapture(0)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

landmarks_to_model=[]
count=1 
ret=True
while ret:
    ret, frame = cap.read()
    
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        if count % 5 == 0:
            landmarks_to_model.append(extract_landmarks_to_np(holistic.process(frame)))

    if count % 100 == 0:
        np.array(landmarks_to_model)
        if landmarks_to_model is None:
            exit()
        pred_probs = model_to_test.predict(np.expand_dims(landmarks_to_model, axis=0))
        if pred_probs[pred_probs > threshold].any():
            pred = np.argmax(pred_probs)
            pred_label = [k for k, v in labels.items() if v == pred]
            print(pred_label)

        landmarks_to_model=[]

    count += 1
    # Show to screen
    cv2.imshow('OpenCV Feed', frame)

    # Break gracefully
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
